<a href="https://colab.research.google.com/github/Shubham04-oss/Medical-Record/blob/main/Patient_Record.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import pandas as pd
import random
from datetime import datetime, timedelta
import json

# Medical data for realistic generation
FIRST_NAMES = ['John', 'Jane', 'Michael', 'Sarah', 'David', 'Emily', 'Robert', 'Lisa', 'James', 'Maria', 'William', 'Jennifer', 'Richard', 'Linda', 'Joseph', 'Patricia', 'Thomas', 'Barbara', 'Christopher', 'Elizabeth']
LAST_NAMES = ['Smith', 'Johnson', 'Williams', 'Brown', 'Jones', 'Garcia', 'Miller', 'Davis', 'Rodriguez', 'Martinez', 'Hernandez', 'Lopez', 'Gonzalez', 'Wilson', 'Anderson', 'Thomas', 'Taylor', 'Moore', 'Jackson', 'Martin']

MEDICATIONS = [
    {'name': 'Lisinopril', 'dosage': '10mg', 'frequency': 'Once daily', 'condition': 'Hypertension'},
    {'name': 'Metformin', 'dosage': '500mg', 'frequency': 'Twice daily', 'condition': 'Diabetes'},
    {'name': 'Atorvastatin', 'dosage': '20mg', 'frequency': 'Once daily', 'condition': 'High Cholesterol'},
    {'name': 'Omeprazole', 'dosage': '20mg', 'frequency': 'Once daily', 'condition': 'GERD'},
    {'name': 'Ibuprofen', 'dosage': '400mg', 'frequency': 'As needed', 'condition': 'Pain'},
    {'name': 'Amoxicillin', 'dosage': '500mg', 'frequency': 'Three times daily', 'condition': 'Infection'},
    {'name': 'Levothyroxine', 'dosage': '50mcg', 'frequency': 'Once daily', 'condition': 'Hypothyroidism'},
    {'name': 'Amlodipine', 'dosage': '5mg', 'frequency': 'Once daily', 'condition': 'Hypertension'},
    {'name': 'Simvastatin', 'dosage': '40mg', 'frequency': 'Once daily', 'condition': 'High Cholesterol'},
    {'name': 'Prednisone', 'dosage': '5mg', 'frequency': 'Once daily', 'condition': 'Inflammation'}
]

PROCEDURES = [
    {'name': 'Appendectomy', 'type': 'Surgery', 'duration': '2-3 hours', 'recovery': '2-4 weeks'},
    {'name': 'Knee Replacement', 'type': 'Surgery', 'duration': '1-2 hours', 'recovery': '3-6 months'},
    {'name': 'Colonoscopy', 'type': 'Diagnostic', 'duration': '30-60 minutes', 'recovery': '1 day'},
    {'name': 'Cataract Surgery', 'type': 'Surgery', 'duration': '15-30 minutes', 'recovery': '1-2 weeks'},
    {'name': 'Hip Replacement', 'type': 'Surgery', 'duration': '1-2 hours', 'recovery': '3-6 months'},
    {'name': 'Gallbladder Removal', 'type': 'Surgery', 'duration': '1-2 hours', 'recovery': '1-2 weeks'},
    {'name': 'Endoscopy', 'type': 'Diagnostic', 'duration': '15-30 minutes', 'recovery': '1 day'},
    {'name': 'Hernia Repair', 'type': 'Surgery', 'duration': '30-60 minutes', 'recovery': '1-3 weeks'},
    {'name': 'Tonsillectomy', 'type': 'Surgery', 'duration': '30-45 minutes', 'recovery': '1-2 weeks'},
    {'name': 'MRI Scan', 'type': 'Diagnostic', 'duration': '30-60 minutes', 'recovery': 'None'}
]

CONDITIONS = [
    'Hypertension', 'Diabetes Type 2', 'High Cholesterol', 'Asthma', 'Depression',
    'Anxiety', 'Arthritis', 'GERD', 'Hypothyroidism', 'Obesity', 'Sleep Apnea',
    'Chronic Pain', 'Migraine', 'Allergies', 'Heart Disease'
]

def create_database():
    """Create SQLite database with medical record tables"""
    conn = sqlite3.connect('medical_records.db')
    cursor = conn.cursor()

    # Create patients table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS patients (
        patient_id TEXT PRIMARY KEY,
        first_name TEXT,
        last_name TEXT,
        date_of_birth DATE,
        gender TEXT,
        phone TEXT,
        email TEXT,
        address TEXT,
        insurance_id TEXT,
        emergency_contact TEXT,
        created_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    ''')

    # Create medications table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS medications (
        medication_id INTEGER PRIMARY KEY AUTOINCREMENT,
        patient_id TEXT,
        medication_name TEXT,
        dosage TEXT,
        frequency TEXT,
        start_date DATE,
        end_date DATE,
        prescribing_doctor TEXT,
        condition_treated TEXT,
        status TEXT,
        FOREIGN KEY (patient_id) REFERENCES patients(patient_id)
    )
    ''')

    # Create procedures table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS procedures (
        procedure_id INTEGER PRIMARY KEY AUTOINCREMENT,
        patient_id TEXT,
        procedure_name TEXT,
        procedure_type TEXT,
        procedure_date DATE,
        performing_doctor TEXT,
        hospital TEXT,
        notes TEXT,
        outcome TEXT,
        cost REAL,
        FOREIGN KEY (patient_id) REFERENCES patients(patient_id)
    )
    ''')

    # Create conditions table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS conditions (
        condition_id INTEGER PRIMARY KEY AUTOINCREMENT,
        patient_id TEXT,
        condition_name TEXT,
        diagnosis_date DATE,
        severity TEXT,
        status TEXT,
        notes TEXT,
        FOREIGN KEY (patient_id) REFERENCES patients(patient_id)
    )
    ''')

    # Create visits table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS visits (
        visit_id INTEGER PRIMARY KEY AUTOINCREMENT,
        patient_id TEXT,
        visit_date DATE,
        visit_type TEXT,
        doctor TEXT,
        department TEXT,
        chief_complaint TEXT,
        diagnosis TEXT,
        treatment_plan TEXT,
        follow_up_date DATE,
        FOREIGN KEY (patient_id) REFERENCES patients(patient_id)
    )
    ''')

    conn.commit()
    return conn

def generate_patient_data(num_patients=30):
    """Generate realistic patient data"""
    patients = []
    medications = []
    procedures = []
    conditions = []
    visits = []

    for i in range(num_patients):
        # Generate patient
        patient_id = f"P{i+1:03d}"
        first_name = random.choice(FIRST_NAMES)
        last_name = random.choice(LAST_NAMES)

        # Generate age between 25-80
        age = random.randint(25, 80)
        birth_date = datetime.now() - timedelta(days=age*365 + random.randint(-180, 180))

        patient = {
            'patient_id': patient_id,
            'first_name': first_name,
            'last_name': last_name,
            'date_of_birth': birth_date.strftime('%Y-%m-%d'),
            'gender': random.choice(['Male', 'Female']),
            'phone': f"({random.randint(100, 999)}) {random.randint(100, 999)}-{random.randint(1000, 9999)}",
            'email': f"{first_name.lower()}.{last_name.lower()}@email.com",
            'address': f"{random.randint(100, 9999)} {random.choice(['Main', 'Oak', 'First', 'Second', 'Park', 'Elm'])} St",
            'insurance_id': f"INS{random.randint(100000, 999999)}",
            'emergency_contact': f"{random.choice(FIRST_NAMES)} {random.choice(LAST_NAMES)}"
        }
        patients.append(patient)

        # Generate 2-5 medications per patient
        num_meds = random.randint(2, 5)
        patient_meds = random.sample(MEDICATIONS, min(num_meds, len(MEDICATIONS)))

        for med in patient_meds:
            start_date = datetime.now() - timedelta(days=random.randint(30, 1095))  # 1 month to 3 years ago
            medication = {
                'patient_id': patient_id,
                'medication_name': med['name'],
                'dosage': med['dosage'],
                'frequency': med['frequency'],
                'start_date': start_date.strftime('%Y-%m-%d'),
                'end_date': None if random.random() > 0.3 else (start_date + timedelta(days=random.randint(30, 365))).strftime('%Y-%m-%d'),
                'prescribing_doctor': f"Dr. {random.choice(LAST_NAMES)}",
                'condition_treated': med['condition'],
                'status': random.choice(['Active', 'Discontinued', 'Completed'])
            }
            medications.append(medication)

        # Generate 0-3 procedures per patient
        num_procedures = random.randint(0, 3)
        if num_procedures > 0:
            patient_procedures = random.sample(PROCEDURES, min(num_procedures, len(PROCEDURES)))

            for proc in patient_procedures:
                procedure_date = datetime.now() - timedelta(days=random.randint(30, 1825))  # 1 month to 5 years ago
                procedure = {
                    'patient_id': patient_id,
                    'procedure_name': proc['name'],
                    'procedure_type': proc['type'],
                    'procedure_date': procedure_date.strftime('%Y-%m-%d'),
                    'performing_doctor': f"Dr. {random.choice(LAST_NAMES)}",
                    'hospital': f"{random.choice(['City', 'General', 'Memorial', 'Regional', 'University'])} Hospital",
                    'notes': f"Successful {proc['name'].lower()}. Recovery time: {proc['recovery']}",
                    'outcome': random.choice(['Successful', 'Completed', 'Good outcome']),
                    'cost': random.randint(1000, 50000)
                }
                procedures.append(procedure)

        # Generate 1-4 conditions per patient
        num_conditions = random.randint(1, 4)
        patient_conditions = random.sample(CONDITIONS, min(num_conditions, len(CONDITIONS)))

        for condition in patient_conditions:
            diagnosis_date = datetime.now() - timedelta(days=random.randint(30, 2190))  # 1 month to 6 years ago
            cond = {
                'patient_id': patient_id,
                'condition_name': condition,
                'diagnosis_date': diagnosis_date.strftime('%Y-%m-%d'),
                'severity': random.choice(['Mild', 'Moderate', 'Severe']),
                'status': random.choice(['Active', 'Resolved', 'Managed', 'Chronic']),
                'notes': f"Patient diagnosed with {condition}"
            }
            conditions.append(cond)

        # Generate 1-5 visits per patient
        num_visits = random.randint(1, 5)
        for v in range(num_visits):
            visit_date = datetime.now() - timedelta(days=random.randint(1, 365))
            visit = {
                'patient_id': patient_id,
                'visit_date': visit_date.strftime('%Y-%m-%d'),
                'visit_type': random.choice(['Regular Checkup', 'Follow-up', 'Emergency', 'Consultation', 'Specialist Visit']),
                'doctor': f"Dr. {random.choice(LAST_NAMES)}",
                'department': random.choice(['Internal Medicine', 'Cardiology', 'Emergency', 'Orthopedics', 'Family Medicine']),
                'chief_complaint': random.choice(['Chest pain', 'Shortness of breath', 'Routine checkup', 'Follow-up visit', 'Joint pain']),
                'diagnosis': random.choice(CONDITIONS),
                'treatment_plan': 'Continue current medications, follow up in 3 months',
                'follow_up_date': (visit_date + timedelta(days=random.randint(30, 180))).strftime('%Y-%m-%d')
            }
            visits.append(visit)

    return patients, medications, procedures, conditions, visits

def insert_data_to_database(conn, patients, medications, procedures, conditions, visits):
    """Insert generated data into database"""
    cursor = conn.cursor()

    # Insert patients
    for patient in patients:
        cursor.execute('''
        INSERT INTO patients (patient_id, first_name, last_name, date_of_birth, gender,
                            phone, email, address, insurance_id, emergency_contact)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (patient['patient_id'], patient['first_name'], patient['last_name'],
              patient['date_of_birth'], patient['gender'], patient['phone'],
              patient['email'], patient['address'], patient['insurance_id'],
              patient['emergency_contact']))

    # Insert medications
    for med in medications:
        cursor.execute('''
        INSERT INTO medications (patient_id, medication_name, dosage, frequency,
                               start_date, end_date, prescribing_doctor, condition_treated, status)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (med['patient_id'], med['medication_name'], med['dosage'], med['frequency'],
              med['start_date'], med['end_date'], med['prescribing_doctor'],
              med['condition_treated'], med['status']))

    # Insert procedures
    for proc in procedures:
        cursor.execute('''
        INSERT INTO procedures (patient_id, procedure_name, procedure_type, procedure_date,
                              performing_doctor, hospital, notes, outcome, cost)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (proc['patient_id'], proc['procedure_name'], proc['procedure_type'],
              proc['procedure_date'], proc['performing_doctor'], proc['hospital'],
              proc['notes'], proc['outcome'], proc['cost']))

    # Insert conditions
    for cond in conditions:
        cursor.execute('''
        INSERT INTO conditions (patient_id, condition_name, diagnosis_date,
                              severity, status, notes)
        VALUES (?, ?, ?, ?, ?, ?)
        ''', (cond['patient_id'], cond['condition_name'], cond['diagnosis_date'],
              cond['severity'], cond['status'], cond['notes']))

    # Insert visits
    for visit in visits:
        cursor.execute('''
        INSERT INTO visits (patient_id, visit_date, visit_type, doctor, department,
                          chief_complaint, diagnosis, treatment_plan, follow_up_date)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (visit['patient_id'], visit['visit_date'], visit['visit_type'], visit['doctor'],
              visit['department'], visit['chief_complaint'], visit['diagnosis'],
              visit['treatment_plan'], visit['follow_up_date']))

    conn.commit()
    print("✅ All data inserted successfully!")

def get_patient_profile(conn, patient_id):
    """Retrieve complete patient profile"""
    cursor = conn.cursor()

    # Get patient basic info
    cursor.execute("SELECT * FROM patients WHERE patient_id = ?", (patient_id,))
    patient = cursor.fetchone()

    if not patient:
        return None

    # Get medications
    cursor.execute("SELECT * FROM medications WHERE patient_id = ?", (patient_id,))
    medications = cursor.fetchall()

    # Get procedures
    cursor.execute("SELECT * FROM procedures WHERE patient_id = ?", (patient_id,))
    procedures = cursor.fetchall()

    # Get conditions
    cursor.execute("SELECT * FROM conditions WHERE patient_id = ?", (patient_id,))
    conditions = cursor.fetchall()

    # Get visits
    cursor.execute("SELECT * FROM visits WHERE patient_id = ? ORDER BY visit_date DESC", (patient_id,))
    visits = cursor.fetchall()

    return {
        'patient': patient,
        'medications': medications,
        'procedures': procedures,
        'conditions': conditions,
        'visits': visits
    }

# Main execution
if __name__ == "__main__":
    print("🏥 Creating Medical Records Database...")

    # Create database
    conn = create_database()
    print("✅ Database created successfully!")

    # Generate data
    print("👥 Generating patient data...")
    patients, medications, procedures, conditions, visits = generate_patient_data(30)
    print(f"✅ Generated data for {len(patients)} patients")
    print(f"   📋 {len(medications)} medications")
    print(f"   🏥 {len(procedures)} procedures")
    print(f"   🩺 {len(conditions)} conditions")
    print(f"   📅 {len(visits)} visits")

    # Insert into database
    print("\n💾 Inserting data into database...")
    insert_data_to_database(conn, patients, medications, procedures, conditions, visits)

    # Test retrieval
    print("\n🔍 Testing patient profile retrieval...")
    profile = get_patient_profile(conn, "P001")
    if profile:
        patient = profile['patient']
        print(f"\n📋 Patient Profile: {patient[1]} {patient[2]} (ID: {patient[0]})")
        print(f"   Age: {patient[3]} | Gender: {patient[4]}")
        print(f"   📞 Phone: {patient[5]}")
        print(f"   💊 Active Medications: {len(profile['medications'])}")
        print(f"   🏥 Procedures: {len(profile['procedures'])}")
        print(f"   🩺 Conditions: {len(profile['conditions'])}")
        print(f"   📅 Visits: {len(profile['visits'])}")

    conn.close()
    print("\n🎉 Medical records database setup complete!")
    print("Ready for ML processing and API development!")

🏥 Creating Medical Records Database...
✅ Database created successfully!
👥 Generating patient data...
✅ Generated data for 30 patients
   📋 107 medications
   🏥 53 procedures
   🩺 70 conditions
   📅 99 visits

💾 Inserting data into database...
✅ All data inserted successfully!

🔍 Testing patient profile retrieval...

📋 Patient Profile: David Davis (ID: P001)
   Age: 1995-06-13 | Gender: Female
   📞 Phone: (484) 272-2489
   💊 Active Medications: 5
   🏥 Procedures: 2
   🩺 Conditions: 2
   📅 Visits: 5

🎉 Medical records database setup complete!
Ready for ML processing and API development!


In [2]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

def explore_medical_data():
    """Explore and visualize our generated medical data"""
    conn = sqlite3.connect('medical_records.db')

    print("🔍 EXPLORING YOUR MEDICAL DATABASE")
    print("=" * 50)

    # 1. Database Overview
    print("\n📊 DATABASE OVERVIEW:")
    tables = ['patients', 'medications', 'procedures', 'conditions', 'visits']
    for table in tables:
        count = pd.read_sql_query(f"SELECT COUNT(*) as count FROM {table}", conn).iloc[0]['count']
        print(f"   {table.upper()}: {count} records")

    # 2. Patient Demographics
    print("\n👥 PATIENT DEMOGRAPHICS:")
    patients_df = pd.read_sql_query("SELECT * FROM patients", conn)
    print(f"   Total Patients: {len(patients_df)}")
    print(f"   Gender Distribution:")
    gender_counts = patients_df['gender'].value_counts()
    for gender, count in gender_counts.items():
        print(f"     {gender}: {count}")

    # 3. Most Common Medications
    print("\n💊 MOST COMMON MEDICATIONS:")
    meds_df = pd.read_sql_query("SELECT medication_name, COUNT(*) as count FROM medications GROUP BY medication_name ORDER BY count DESC LIMIT 10", conn)
    for _, row in meds_df.iterrows():
        print(f"   {row['medication_name']}: {row['count']} patients")

    # 4. Most Common Conditions
    print("\n🩺 MOST COMMON CONDITIONS:")
    conditions_df = pd.read_sql_query("SELECT condition_name, COUNT(*) as count FROM conditions GROUP BY condition_name ORDER BY count DESC LIMIT 10", conn)
    for _, row in conditions_df.iterrows():
        print(f"   {row['condition_name']}: {row['count']} patients")

    # 5. Procedure Statistics
    print("\n🏥 PROCEDURE STATISTICS:")
    procedures_df = pd.read_sql_query("SELECT procedure_name, COUNT(*) as count FROM procedures GROUP BY procedure_name ORDER BY count DESC", conn)
    for _, row in procedures_df.iterrows():
        print(f"   {row['procedure_name']}: {row['count']} patients")

    # 6. Recent Activity
    print("\n📅 RECENT ACTIVITY (Last 30 days):")
    recent_visits = pd.read_sql_query("""
        SELECT COUNT(*) as count FROM visits
        WHERE visit_date >= date('now', '-30 days')
    """, conn).iloc[0]['count']
    print(f"   Recent Visits: {recent_visits}")

    conn.close()
    return True

def show_sample_patients(num_patients=5):
    """Show detailed profiles of sample patients"""
    conn = sqlite3.connect('medical_records.db')

    print(f"\n🏥 SAMPLE PATIENT PROFILES:")
    print("=" * 60)

    for i in range(1, num_patients + 1):
        patient_id = f"P{i:03d}"

        # Get patient info
        patient = pd.read_sql_query("SELECT * FROM patients WHERE patient_id = ?", conn, params=[patient_id])
        if patient.empty:
            continue

        p = patient.iloc[0]
        print(f"\n👤 PATIENT {patient_id}: {p['first_name']} {p['last_name']}")
        print(f"   📧 {p['email']} | 📞 {p['phone']}")
        print(f"   🎂 Born: {p['date_of_birth']} | ⚤ {p['gender']}")

        # Get medications
        meds = pd.read_sql_query("SELECT * FROM medications WHERE patient_id = ?", conn, params=[patient_id])
        print(f"\n   💊 MEDICATIONS ({len(meds)}):")
        for _, med in meds.iterrows():
            status_emoji = "✅" if med['status'] == 'Active' else "⏸️"
            print(f"      {status_emoji} {med['medication_name']} {med['dosage']} - {med['frequency']}")
            print(f"         For: {med['condition_treated']} | Status: {med['status']}")

        # Get conditions
        conditions = pd.read_sql_query("SELECT * FROM conditions WHERE patient_id = ?", conn, params=[patient_id])
        print(f"\n   🩺 CONDITIONS ({len(conditions)}):")
        for _, cond in conditions.iterrows():
            severity_emoji = "🔴" if cond['severity'] == 'Severe' else "🟡" if cond['severity'] == 'Moderate' else "🟢"
            print(f"      {severity_emoji} {cond['condition_name']} ({cond['severity']}) - {cond['status']}")

        # Get procedures
        procedures = pd.read_sql_query("SELECT * FROM procedures WHERE patient_id = ?", conn, params=[patient_id])
        if not procedures.empty:
            print(f"\n   🏥 PROCEDURES ({len(procedures)}):")
            for _, proc in procedures.iterrows():
                print(f"      🔸 {proc['procedure_name']} ({proc['procedure_date']})")
                print(f"         Doctor: {proc['performing_doctor']} | Outcome: {proc['outcome']}")

        # Get recent visits
        visits = pd.read_sql_query("SELECT * FROM visits WHERE patient_id = ? ORDER BY visit_date DESC LIMIT 3", conn, params=[patient_id])
        print(f"\n   📅 RECENT VISITS ({len(visits)}):")
        for _, visit in visits.iterrows():
            print(f"      📍 {visit['visit_date']}: {visit['visit_type']}")
            print(f"         Chief Complaint: {visit['chief_complaint']}")

        print("-" * 60)

    conn.close()

def search_patient_demo():
    """Demo of patient search functionality"""
    conn = sqlite3.connect('medical_records.db')

    print("\n🔍 PATIENT SEARCH DEMO:")
    print("=" * 40)

    # Search by name
    print("\n1️⃣ SEARCH BY NAME (John):")
    results = pd.read_sql_query("""
        SELECT patient_id, first_name, last_name, gender, date_of_birth
        FROM patients
        WHERE first_name LIKE '%John%' OR last_name LIKE '%John%'
    """, conn)

    for _, patient in results.iterrows():
        print(f"   📋 {patient['patient_id']}: {patient['first_name']} {patient['last_name']}")
        print(f"      Born: {patient['date_of_birth']} | Gender: {patient['gender']}")

    # Search by condition
    print("\n2️⃣ PATIENTS WITH DIABETES:")
    diabetes_patients = pd.read_sql_query("""
        SELECT DISTINCT p.patient_id, p.first_name, p.last_name, c.condition_name
        FROM patients p
        JOIN conditions c ON p.patient_id = c.patient_id
        WHERE c.condition_name LIKE '%Diabetes%'
    """, conn)

    for _, patient in diabetes_patients.iterrows():
        print(f"   🩺 {patient['patient_id']}: {patient['first_name']} {patient['last_name']}")

    # Search by medication
    print("\n3️⃣ PATIENTS ON LISINOPRIL:")
    lisinopril_patients = pd.read_sql_query("""
        SELECT DISTINCT p.patient_id, p.first_name, p.last_name, m.medication_name, m.status
        FROM patients p
        JOIN medications m ON p.patient_id = m.patient_id
        WHERE m.medication_name = 'Lisinopril'
    """, conn)

    for _, patient in lisinopril_patients.iterrows():
        status_emoji = "✅" if patient['status'] == 'Active' else "⏸️"
        print(f"   💊 {patient['patient_id']}: {patient['first_name']} {patient['last_name']} {status_emoji}")

    conn.close()

# Run all explorations
print("🚀 EXPLORING YOUR MEDICAL DATABASE...")
explore_medical_data()
show_sample_patients(3)  # Show 3 detailed patient profiles
search_patient_demo()

print("\n\n🎉 DATA EXPLORATION COMPLETE!")
print("✅ Your database is ready for ML processing!")
print("✅ Next: Build patient search API and ML components!")


🚀 EXPLORING YOUR MEDICAL DATABASE...
🔍 EXPLORING YOUR MEDICAL DATABASE

📊 DATABASE OVERVIEW:
   PATIENTS: 30 records
   MEDICATIONS: 107 records
   PROCEDURES: 53 records
   CONDITIONS: 70 records
   VISITS: 99 records

👥 PATIENT DEMOGRAPHICS:
   Total Patients: 30
   Gender Distribution:
     Female: 19
     Male: 11

💊 MOST COMMON MEDICATIONS:
   Levothyroxine: 14 patients
   Ibuprofen: 12 patients
   Simvastatin: 11 patients
   Prednisone: 11 patients
   Omeprazole: 11 patients
   Amoxicillin: 11 patients
   Amlodipine: 11 patients
   Metformin: 10 patients
   Lisinopril: 10 patients
   Atorvastatin: 6 patients

🩺 MOST COMMON CONDITIONS:
   Heart Disease: 8 patients
   Diabetes Type 2: 8 patients
   Sleep Apnea: 7 patients
   Anxiety: 7 patients
   Allergies: 6 patients
   Migraine: 5 patients
   Depression: 5 patients
   Chronic Pain: 5 patients
   GERD: 4 patients
   Arthritis: 4 patients

🏥 PROCEDURE STATISTICS:
   MRI Scan: 12 patients
   Tonsillectomy: 8 patients
   Colonoscopy

In [3]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import re
from datetime import datetime, timedelta
from collections import Counter, defaultdict
import warnings
warnings.filterwarnings('ignore')

class MedicalMLEngine:
    def __init__(self, db_path='medical_records.db'):
        self.db_path = db_path
        self.vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
        self.scaler = StandardScaler()

    def connect_db(self):
        return sqlite3.connect(self.db_path)

    def patient_matching(self, threshold=0.8):
        """
        Find potential duplicate patients using ML similarity matching
        """
        print("🔍 ML PATIENT MATCHING - Finding Potential Duplicates...")

        conn = self.connect_db()
        patients_df = pd.read_sql_query("""
            SELECT patient_id, first_name, last_name, date_of_birth,
                   phone, email, address FROM patients
        """, conn)
        conn.close()

        # Create patient profiles for matching
        patients_df['full_profile'] = (
            patients_df['first_name'] + ' ' +
            patients_df['last_name'] + ' ' +
            patients_df['date_of_birth'] + ' ' +
            patients_df['phone'].fillna('') + ' ' +
            patients_df['address'].fillna('')
        )

        # Vectorize patient profiles
        profile_vectors = self.vectorizer.fit_transform(patients_df['full_profile'])

        # Calculate similarity matrix
        similarity_matrix = cosine_similarity(profile_vectors)

        # Find potential duplicates
        duplicates = []
        for i in range(len(patients_df)):
            for j in range(i+1, len(patients_df)):
                similarity = similarity_matrix[i][j]
                if similarity > threshold:
                    duplicates.append({
                        'patient1_id': patients_df.iloc[i]['patient_id'],
                        'patient1_name': f"{patients_df.iloc[i]['first_name']} {patients_df.iloc[i]['last_name']}",
                        'patient2_id': patients_df.iloc[j]['patient_id'],
                        'patient2_name': f"{patients_df.iloc[j]['first_name']} {patients_df.iloc[j]['last_name']}",
                        'similarity_score': round(similarity, 3)
                    })

        print(f"✅ Found {len(duplicates)} potential duplicate pairs")
        return duplicates

    def extract_medical_entities(self, text):
        """
        Extract medical entities from clinical text using rule-based NLP
        """
        # Common medical patterns
        medication_patterns = [
            r'\b(lisinopril|metformin|atorvastatin|omeprazole|ibuprofen|amoxicillin)\b',
            r'\b\w+\s+\d+mg\b',  # Drug with dosage
            r'\b\w+\s+twice\s+daily\b'  # Drug with frequency
        ]

        condition_patterns = [
            r'\b(diabetes|hypertension|high\s+blood\s+pressure|heart\s+disease)\b',
            r'\b(depression|anxiety|asthma|arthritis|migraine)\b'
        ]

        procedure_patterns = [
            r'\b(surgery|operation|procedure|biopsy|scan|x-ray|mri|ct)\b',
            r'\b(appendectomy|colonoscopy|endoscopy)\b'
        ]

        entities = {
            'medications': [],
            'conditions': [],
            'procedures': []
        }

        text_lower = text.lower()

        # Extract medications
        for pattern in medication_patterns:
            matches = re.findall(pattern, text_lower, re.IGNORECASE)
            entities['medications'].extend(matches)

        # Extract conditions
        for pattern in condition_patterns:
            matches = re.findall(pattern, text_lower, re.IGNORECASE)
            entities['conditions'].extend(matches)

        # Extract procedures
        for pattern in procedure_patterns:
            matches = re.findall(pattern, text_lower, re.IGNORECASE)
            entities['procedures'].extend(matches)

        return entities

    def patient_risk_assessment(self):
        """
        Assess patient risk levels using ML clustering and rule-based scoring
        """
        print("⚠️ ML RISK ASSESSMENT - Analyzing Patient Risk Levels...")

        conn = self.connect_db()

        # Get comprehensive patient data
        query = """
        SELECT p.patient_id, p.first_name, p.last_name, p.date_of_birth,
               COUNT(DISTINCT m.medication_id) as medication_count,
               COUNT(DISTINCT pr.procedure_id) as procedure_count,
               COUNT(DISTINCT c.condition_id) as condition_count,
               COUNT(DISTINCT v.visit_id) as visit_count,
               GROUP_CONCAT(DISTINCT c.condition_name) as conditions,
               GROUP_CONCAT(DISTINCT c.severity) as severities
        FROM patients p
        LEFT JOIN medications m ON p.patient_id = m.patient_id
        LEFT JOIN procedures pr ON p.patient_id = pr.patient_id
        LEFT JOIN conditions c ON p.patient_id = c.patient_id
        LEFT JOIN visits v ON p.patient_id = v.patient_id
        GROUP BY p.patient_id
        """

        risk_df = pd.read_sql_query(query, conn)
        conn.close()

        # Calculate age
        risk_df['age'] = risk_df['date_of_birth'].apply(
            lambda x: (datetime.now() - datetime.strptime(x, '%Y-%m-%d')).days // 365
        )

        # Risk scoring based on multiple factors
        risk_df['risk_score'] = 0

        # Age risk (higher age = higher risk)
        risk_df['risk_score'] += np.where(risk_df['age'] > 65, 20, 0)
        risk_df['risk_score'] += np.where(risk_df['age'] > 75, 10, 0)

        # Medication count risk
        risk_df['risk_score'] += risk_df['medication_count'] * 5

        # Condition severity risk
        risk_df['severe_conditions'] = risk_df['severities'].fillna('').str.count('Severe')
        risk_df['risk_score'] += risk_df['severe_conditions'] * 15

        # High-risk conditions
        high_risk_conditions = ['diabetes', 'heart disease', 'hypertension']
        for condition in high_risk_conditions:
            risk_df['risk_score'] += risk_df['conditions'].fillna('').str.lower().str.contains(condition).astype(int) * 10

        # Frequent visits (could indicate health issues)
        risk_df['risk_score'] += np.where(risk_df['visit_count'] > 5, 15, 0)

        # Categorize risk levels
        risk_df['risk_level'] = pd.cut(
            risk_df['risk_score'],
            bins=[0, 25, 50, 75, 100],
            labels=['Low', 'Medium', 'High', 'Critical'],
            include_lowest=True
        )

        risk_summary = risk_df['risk_level'].value_counts().to_dict()
        print(f"✅ Risk Assessment Complete:")
        for level, count in risk_summary.items():
            emoji = "🟢" if level == 'Low' else "🟡" if level == 'Medium' else "🔴" if level == 'High' else "🆘"
            print(f"   {emoji} {level} Risk: {count} patients")

        return risk_df[['patient_id', 'first_name', 'last_name', 'age', 'risk_score', 'risk_level']]

    def smart_patient_search(self, query, search_type='fuzzy'):
        """
        Intelligent patient search using ML similarity matching
        """
        conn = self.connect_db()

        if search_type == 'fuzzy':
            # Get all patients with comprehensive info
            patients_df = pd.read_sql_query("""
                SELECT p.patient_id, p.first_name, p.last_name, p.date_of_birth,
                       p.phone, p.email,
                       GROUP_CONCAT(DISTINCT c.condition_name) as conditions,
                       GROUP_CONCAT(DISTINCT m.medication_name) as medications
                FROM patients p
                LEFT JOIN conditions c ON p.patient_id = c.patient_id
                LEFT JOIN medications m ON p.patient_id = m.patient_id
                GROUP BY p.patient_id
            """, conn)

            # Create searchable profile
            patients_df['searchable_profile'] = (
                patients_df['first_name'].fillna('') + ' ' +
                patients_df['last_name'].fillna('') + ' ' +
                patients_df['conditions'].fillna('') + ' ' +
                patients_df['medications'].fillna('')
            )

            # Vectorize profiles and query
            all_text = list(patients_df['searchable_profile']) + [query]
            vectors = self.vectorizer.fit_transform(all_text)

            # Calculate similarity with query
            query_vector = vectors[-1]
            patient_vectors = vectors[:-1]
            similarities = cosine_similarity(query_vector, patient_vectors)[0]

            # Get top matches
            patients_df['similarity'] = similarities
            results = patients_df[patients_df['similarity'] > 0.1].sort_values('similarity', ascending=False)

        else:
            # Exact search
            results = pd.read_sql_query(f"""
                SELECT patient_id, first_name, last_name, date_of_birth, phone, email
                FROM patients
                WHERE first_name LIKE '%{query}%'
                   OR last_name LIKE '%{query}%'
                   OR patient_id LIKE '%{query}%'
            """, conn)

        conn.close()
        return results

    def medication_interaction_checker(self, patient_id):
        """
        Check for potential drug interactions using rule-based system
        """
        conn = self.connect_db()

        medications_df = pd.read_sql_query("""
            SELECT medication_name, dosage, status FROM medications
            WHERE patient_id = ? AND status = 'Active'
        """, conn, params=[patient_id])

        conn.close()

        # Known drug interactions (simplified)
        interactions = {
            ('lisinopril', 'ibuprofen'): 'May reduce effectiveness of blood pressure medication',
            ('metformin', 'prednisone'): 'May affect blood sugar control',
            ('atorvastatin', 'amoxicillin'): 'Monitor liver function'
        }

        active_meds = [med.lower() for med in medications_df['medication_name']]
        found_interactions = []

        for (drug1, drug2), warning in interactions.items():
            if drug1 in active_meds and drug2 in active_meds:
                found_interactions.append({
                    'drug1': drug1,
                    'drug2': drug2,
                    'warning': warning
                })

        return found_interactions

    def generate_patient_timeline(self, patient_id):
        """
        Generate chronological medical timeline using ML-enhanced sorting
        """
        conn = self.connect_db()

        # Get all events for patient
        events = []

        # Medications
        meds = pd.read_sql_query("""
            SELECT 'Medication' as event_type, medication_name as description,
                   start_date as event_date, 'Started ' || medication_name as detail
            FROM medications WHERE patient_id = ?
        """, conn, params=[patient_id])
        events.append(meds)

        # Procedures
        procs = pd.read_sql_query("""
            SELECT 'Procedure' as event_type, procedure_name as description,
                   procedure_date as event_date, notes as detail
            FROM procedures WHERE patient_id = ?
        """, conn, params=[patient_id])
        events.append(procs)

        # Visits
        visits = pd.read_sql_query("""
            SELECT 'Visit' as event_type, visit_type as description,
                   visit_date as event_date, chief_complaint as detail
            FROM visits WHERE patient_id = ?
        """, conn, params=[patient_id])
        events.append(visits)

        # Conditions
        conditions = pd.read_sql_query("""
            SELECT 'Diagnosis' as event_type, condition_name as description,
                   diagnosis_date as event_date, 'Diagnosed with ' || condition_name as detail
            FROM conditions WHERE patient_id = ?
        """, conn, params=[patient_id])
        events.append(conditions)

        conn.close()

        # Combine all events
        if events:
            timeline = pd.concat([e for e in events if not e.empty], ignore_index=True)
            timeline['event_date'] = pd.to_datetime(timeline['event_date'])
            timeline = timeline.sort_values('event_date', ascending=False)
            return timeline

        return pd.DataFrame()

# Initialize and demonstrate ML engine
def demo_ml_engine():
    print("🤖 INITIALIZING MEDICAL ML ENGINE...")
    print("=" * 50)

    ml_engine = MedicalMLEngine()

    # 1. Patient Matching Demo
    duplicates = ml_engine.patient_matching(threshold=0.7)
    if duplicates:
        print(f"\n🔍 DUPLICATE DETECTION:")
        for dup in duplicates[:3]:  # Show first 3
            print(f"   📋 {dup['patient1_name']} ↔️ {dup['patient2_name']} (Score: {dup['similarity_score']})")

    # 2. Risk Assessment Demo
    risk_results = ml_engine.patient_risk_assessment()
    high_risk = risk_results[risk_results['risk_level'].isin(['High', 'Critical'])]
    print(f"\n⚠️ HIGH-RISK PATIENTS:")
    for _, patient in high_risk.head(5).iterrows():
        emoji = "🔴" if patient['risk_level'] == 'High' else "🆘"
        print(f"   {emoji} {patient['first_name']} {patient['last_name']} (Age: {patient['age']}, Score: {patient['risk_score']})")

    # 3. Smart Search Demo
    print(f"\n🔍 SMART SEARCH DEMO:")
    search_results = ml_engine.smart_patient_search("diabetes medication")
    print(f"   Found {len(search_results)} patients matching 'diabetes medication':")
    for _, patient in search_results.head(3).iterrows():
        similarity = patient.get('similarity', 0)
        print(f"   📋 {patient['first_name']} {patient['last_name']} (Relevance: {similarity:.3f})")

    # 4. Drug Interaction Check
    print(f"\n💊 DRUG INTERACTION CHECK (Patient P001):")
    interactions = ml_engine.medication_interaction_checker('P001')
    if interactions:
        for interaction in interactions:
            print(f"   ⚠️ {interaction['drug1'].title()} + {interaction['drug2'].title()}: {interaction['warning']}")
    else:
        print(f"   ✅ No known interactions found")

    # 5. Patient Timeline
    print(f"\n📅 MEDICAL TIMELINE (Patient P001):")
    timeline = ml_engine.generate_patient_timeline('P001')
    if not timeline.empty:
        for _, event in timeline.head(5).iterrows():
            date_str = event['event_date'].strftime('%Y-%m-%d')
            print(f"   📍 {date_str}: {event['event_type']} - {event['description']}")

    print(f"\n🎉 ML ENGINE DEMO COMPLETE!")
    print(f"✅ All ML components working!")

    return ml_engine

# Run ML engine demo
ml_engine = demo_ml_engine()
print(f"\n🚀 READY FOR API DEVELOPMENT!")

🤖 INITIALIZING MEDICAL ML ENGINE...
🔍 ML PATIENT MATCHING - Finding Potential Duplicates...
✅ Found 0 potential duplicate pairs
⚠️ ML RISK ASSESSMENT - Analyzing Patient Risk Levels...
✅ Risk Assessment Complete:
   🟡 Medium Risk: 17 patients
   🟢 Low Risk: 6 patients
   🔴 High Risk: 6 patients
   🆘 Critical Risk: 1 patients

⚠️ HIGH-RISK PATIENTS:
   🔴 Christopher Thomas (Age: 73, Score: 65)
   🔴 Christopher Brown (Age: 72, Score: 65)
   🔴 John Anderson (Age: 69, Score: 60)
   🔴 Christopher Smith (Age: 79, Score: 55)
   🔴 Jane Smith (Age: 76, Score: 70)

🔍 SMART SEARCH DEMO:
   Found 8 patients matching 'diabetes medication':
   📋 Christopher Brown (Relevance: 0.160)
   📋 Christopher Lopez (Relevance: 0.148)
   📋 Robert Davis (Relevance: 0.147)

💊 DRUG INTERACTION CHECK (Patient P001):
   ✅ No known interactions found

📅 MEDICAL TIMELINE (Patient P001):
   📍 2025-04-18: Procedure - Hernia Repair
   📍 2025-04-03: Visit - Emergency
   📍 2025-03-04: Visit - Emergency
   📍 2024-12-18: Vis

In [4]:
pip install fastapi uvicorn pyngrok


In [12]:
# First install required packages
# !pip install fastapi uvicorn pydantic pyngrok

from fastapi import FastAPI, HTTPException, Query
from fastapi.responses import HTMLResponse, JSONResponse
from pydantic import BaseModel
from typing import List, Optional, Dict, Any
import sqlite3
import pandas as pd
import numpy as np # Import numpy
from datetime import datetime
import uvicorn
from pyngrok import ngrok
import json
import threading # Import threading

# Import our ML engine (assuming it's in the same environment)
# from previous cell: MedicalMLEngine class

app = FastAPI(
    title="Medical Records Management API",
    description="Complete patient medical records system with ML-powered features",
    version="1.0.0"
)

# Initialize ML Engine
ml_engine = MedicalMLEngine()

# Pydantic Models for Request/Response
class PatientBase(BaseModel):
    first_name: str
    last_name: str
    date_of_birth: str
    gender: str
    phone: Optional[str] = None
    email: Optional[str] = None
    address: Optional[str] = None
    insurance_id: Optional[str] = None
    emergency_contact: Optional[str] = None

class PatientCreate(PatientBase):
    pass

class Patient(PatientBase):
    patient_id: str
    created_date: Optional[str] = None

class Medication(BaseModel):
    medication_name: str
    dosage: str
    frequency: str
    start_date: str
    end_date: Optional[str] = None
    prescribing_doctor: str
    condition_treated: str
    status: str

class Procedure(BaseModel):
    procedure_name: str
    procedure_type: str
    procedure_date: str
    performing_doctor: str
    hospital: str
    notes: Optional[str] = None
    outcome: str
    cost: Optional[float] = None

class Condition(BaseModel):
    condition_name: str
    diagnosis_date: str
    severity: str
    status: str
    notes: Optional[str] = None

class Visit(BaseModel):
    visit_date: str
    visit_type: str
    doctor: str
    department: str
    chief_complaint: str
    diagnosis: str
    treatment_plan: str
    follow_up_date: Optional[str] = None

class PatientProfile(BaseModel):
    patient: Patient
    medications: List[Medication]
    procedures: List[Procedure]
    conditions: List[Condition]
    visits: List[Visit]
    risk_assessment: Optional[Dict[str, Any]] = None

class APIResponse(BaseModel):
    status: str
    message: str
    data: Optional[Any] = None
    timestamp: str

# Database connection helper
def get_db_connection():
    return sqlite3.connect('medical_records.db')

# Helper function to create API response
def create_response(status: str, message: str, data: Any = None):
    return APIResponse(
        status=status,
        message=message,
        data=data,
        timestamp=datetime.now().isoformat()
    )

# Helper function to convert numpy types to python native types
def convert_numpy_types(obj):
    """Recursively convert numpy types to Python native types"""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {k: convert_numpy_types(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(item) for item in obj]
    else:
        return obj

# ============================================================================
# CORE PATIENT ENDPOINTS
# ============================================================================

@app.get("/", response_class=HTMLResponse)
async def root():
    """API Documentation and Welcome Page"""
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Medical Records API</title>
        <style>
            body { font-family: Arial, sans-serif; margin: 40px; background: #f5f5f5; }
            .container { background: white; padding: 30px; border-radius: 10px; box-shadow: 0 2px 10px rgba(0,0,0,0.1); }
            h1 { color: #2c5aa0; }
            .endpoint { background: #f8f9fa; padding: 15px; margin: 10px 0; border-left: 4px solid #007bff; }
            .method { font-weight: bold; color: #28a745; }
            .url { color: #6c757d; font-family: monospace; }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>🏥 Medical Records Management API</h1>
            <p>Complete patient medical records system with ML-powered features</p>

            <h2>🚀 Available Endpoints:</h2>

            <div class="endpoint">
                <div class="method">GET</div>
                <div class="url">/patients/{patient_id} - Get complete patient profile</div>
            </div>

            <div class="endpoint">
                <div class="method">GET</div>
                <div class="url">/patients/search/{query} - Smart patient search</div>
            </div>

            <div class="endpoint">
                <div class="method">GET</div>
                <div class="url">/patients/{patient_id}/timeline - Medical timeline</div>
            </div>

            <div class="endpoint">
                <div class="method">GET</div>
                <div class="url">/patients/{patient_id}/risk - Risk assessment</div>
            </div>

            <div class="endpoint">
                <div class="method">GET</div>
                <div class="url">/patients/{patient_id}/interactions - Drug interactions</div>
            </div>

            <div class="endpoint">
                <div class="method">POST</div>
                <div class="url">/patients/ - Add new patient</div>
            </div>

            <div class="endpoint">
                <div class="method">GET</div>
                <div class="url">/stats/ - Database statistics</div>
            </div>

            <h2>📚 Interactive Documentation:</h2>
            <p><a href="/docs" style="color: #007bff;">FastAPI Swagger UI Documentation</a></p>
            <p><a href="/redoc" style="color: #007bff;">ReDoc Documentation</a></p>
        </div>
    </body>
    </html>
    """
    return html_content

@app.get("/patients/{patient_id}", response_model=APIResponse)
async def get_patient_profile(patient_id: str):
    """Get complete patient profile with all medical records"""
    try:
        conn = get_db_connection()

        # Get patient basic info
        patient_df = pd.read_sql_query(
            "SELECT * FROM patients WHERE patient_id = ?",
            conn, params=[patient_id]
        )

        if patient_df.empty:
            raise HTTPException(status_code=404, detail="Patient not found")

        # Get all related records
        medications_df = pd.read_sql_query(
            "SELECT * FROM medications WHERE patient_id = ? ORDER BY start_date DESC",
            conn, params=[patient_id]
        )

        procedures_df = pd.read_sql_query(
            "SELECT * FROM procedures WHERE patient_id = ? ORDER BY procedure_date DESC",
            conn, params=[patient_id]
        )

        conditions_df = pd.read_sql_query(
            "SELECT * FROM conditions WHERE patient_id = ? ORDER BY diagnosis_date DESC",
            conn, params=[patient_id]
        )

        visits_df = pd.read_sql_query(
            "SELECT * FROM visits WHERE patient_id = ? ORDER BY visit_date DESC",
            conn, params=[patient_id]
        )

        conn.close()

        # Build response
        patient_data = {
            "patient": dict(patient_df.iloc[0]),
            "medications": medications_df.to_dict('records'),
            "procedures": procedures_df.to_dict('records'),
            "conditions": conditions_df.to_dict('records'),
            "visits": visits_df.to_dict('records'),
            "summary": {
                "total_medications": len(medications_df),
                "total_procedures": len(procedures_df),
                "total_conditions": len(conditions_df),
                "total_visits": len(visits_df)
            }
        }

        return create_response("success", f"Patient profile retrieved for {patient_id}", patient_data)

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/patients/search/{query}", response_model=APIResponse)
async def search_patients(
    query: str,
    search_type: str = Query("fuzzy", description="Search type: 'fuzzy' or 'exact'"),
    limit: int = Query(10, description="Maximum number of results")
):
    """Smart patient search using ML similarity matching"""
    try:
        results = ml_engine.smart_patient_search(query, search_type)

        if results.empty:
            return create_response("success", "No patients found", [])

        # Limit results
        results = results.head(limit)

        # Convert numpy types in search results
        results_dict = results.to_dict('records')
        converted_results = convert_numpy_types(results_dict)


        search_data = {
            "query": query,
            "search_type": search_type,
            "total_results": len(results),
            "patients": converted_results
        }

        return create_response("success", f"Found {len(results)} patients matching '{query}'", search_data)

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/patients/{patient_id}/timeline", response_model=APIResponse)
async def get_patient_timeline(patient_id: str):
    """Generate chronological medical timeline for patient"""
    try:
        timeline = ml_engine.generate_patient_timeline(patient_id)

        if timeline.empty:
            return create_response("success", "No timeline events found", [])

        # Convert numpy types in timeline
        timeline_dict = timeline.to_dict('records')
        converted_timeline = convert_numpy_types(timeline_dict)


        timeline_data = {
            "patient_id": patient_id,
            "total_events": len(timeline),
            "timeline": converted_timeline
        }

        return create_response("success", f"Timeline generated for {patient_id}", timeline_data)

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/patients/{patient_id}/risk", response_model=APIResponse)
async def get_patient_risk(patient_id: str):
    """Get ML-powered risk assessment for patient"""
    try:
        # Get risk assessment for all patients, filter for specific patient
        risk_df = ml_engine.patient_risk_assessment()
        patient_risk = risk_df[risk_df['patient_id'] == patient_id]

        if patient_risk.empty:
            raise HTTPException(status_code=404, detail="Patient not found")

        risk_data = dict(patient_risk.iloc[0])

        # Convert numpy types in risk data
        converted_risk_data = convert_numpy_types(risk_data)


        return create_response("success", f"Risk assessment for {patient_id}", converted_risk_data)

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/patients/{patient_id}/interactions", response_model=APIResponse)
async def check_drug_interactions(patient_id: str):
    """Check for potential drug interactions"""
    try:
        interactions = ml_engine.medication_interaction_checker(patient_id)

        interaction_data = {
            "patient_id": patient_id,
            "total_interactions": len(interactions),
            "interactions": interactions,
            "risk_level": "High" if interactions else "Low"
        }

        # Convert numpy types in interaction data
        converted_interaction_data = convert_numpy_types(interaction_data)


        return create_response("success", f"Drug interactions checked for {patient_id}", converted_interaction_data)

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# ============================================================================
# DATA MANAGEMENT ENDPOINTS
# ============================================================================

@app.post("/patients/", response_model=APIResponse)
async def create_patient(patient: PatientCreate):
    """Add new patient to the database"""
    try:
        conn = get_db_connection()
        cursor = conn.cursor()

        # Generate new patient ID
        cursor.execute("SELECT COUNT(*) FROM patients")
        count = cursor.fetchone()[0]
        new_patient_id = f"P{count + 1:03d}"

        # Insert new patient
        cursor.execute("""
            INSERT INTO patients (patient_id, first_name, last_name, date_of_birth,
                                gender, phone, email, address, insurance_id, emergency_contact)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            new_patient_id, patient.first_name, patient.last_name,
            patient.date_of_birth, patient.gender, patient.phone,
            patient.email, patient.address, patient.insurance_id,
            patient.emergency_contact
        ))

        conn.commit()
        conn.close()

        new_patient_data = {
            "patient_id": new_patient_id,
            **patient.dict()
        }

        return create_response("success", f"Patient created with ID {new_patient_id}", new_patient_data)

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/stats/", response_model=APIResponse)
async def get_database_stats():
    """Get comprehensive database statistics"""
    try:
        conn = get_db_connection()

        # Get counts for each table
        stats = {}
        tables = ['patients', 'medications', 'procedures', 'conditions', 'visits']

        for table in tables:
            count = pd.read_sql_query(f"SELECT COUNT(*) as count FROM {table}", conn).iloc[0]['count']
            stats[f"total_{table}"] = count

        # Get additional insights
        # Most common medications
        top_meds = pd.read_sql_query("""
            SELECT medication_name, COUNT(*) as count
            FROM medications
            GROUP BY medication_name
            ORDER BY count DESC
            LIMIT 5
        """, conn).to_dict('records')

        # Most common conditions
        top_conditions = pd.read_sql_query("""
            SELECT condition_name, COUNT(*) as count
            FROM conditions
            GROUP BY condition_name
            ORDER BY count DESC
            LIMIT 5
        """, conn).to_dict('records')

        # Gender distribution
        gender_dist = pd.read_sql_query("""
            SELECT gender, COUNT(*) as count
            FROM patients
            GROUP BY gender
        """, conn).to_dict('records')

        conn.close()

        stats_data = {
            "database_overview": stats,
            "insights": {
                "top_medications": top_meds,
                "top_conditions": top_conditions,
                "gender_distribution": gender_dist
            },
            "last_updated": datetime.now().isoformat()
        }

        # Convert numpy types in stats data
        converted_stats_data = convert_numpy_types(stats_data)


        return create_response("success", "Database statistics retrieved", converted_stats_data)

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# ============================================================================
# ML-POWERED ENDPOINTS
# ============================================================================

@app.get("/ml/duplicates", response_model=APIResponse)
async def find_duplicate_patients(threshold: float = Query(0.8, description="Similarity threshold (0.0-1.0)")):
    """Find potential duplicate patients using ML"""
    try:
        duplicates = ml_engine.patient_matching(threshold)

        # Convert numpy types in duplicates data
        converted_duplicates = convert_numpy_types(duplicates)


        duplicate_data = {
            "threshold_used": threshold,
            "total_duplicates": len(converted_duplicates),
            "potential_duplicates": converted_duplicates
        }

        return create_response("success", f"Found {len(converted_duplicates)} potential duplicates", duplicate_data)

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/ml/high-risk-patients", response_model=APIResponse)
async def get_high_risk_patients():
    """Get all high-risk and critical patients"""
    try:
        risk_df = ml_engine.patient_risk_assessment()
        high_risk = risk_df[risk_df['risk_level'].isin(['High', 'Critical'])]

        # Convert numpy types in high-risk data
        high_risk_dict = high_risk.to_dict('records')
        converted_high_risk = convert_numpy_types(high_risk_dict)


        high_risk_data = {
            "total_high_risk": len(converted_high_risk),
            "patients": converted_high_risk
        }

        return create_response("success", f"Found {len(converted_high_risk)} high-risk patients", high_risk_data)

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/patients/all", response_model=APIResponse)
async def get_all_patients(
    limit: int = Query(50, description="Maximum number of patients to return"),
    offset: int = Query(0, description="Number of patients to skip")
):
    """Get all patients with pagination"""
    try:
        conn = get_db_connection()

        patients_df = pd.read_sql_query(f"""
            SELECT patient_id, first_name, last_name, date_of_birth, gender, phone, email
            FROM patients
            ORDER BY patient_id
            LIMIT {limit} OFFSET {offset}
        """, conn)

        total_count = pd.read_sql_query("SELECT COUNT(*) as count FROM patients", conn).iloc[0]['count']

        conn.close()

        # Convert numpy types in patients data
        patients_dict = patients_df.to_dict('records')
        converted_patients = convert_numpy_types(patients_dict)


        patients_data = {
            "total_patients": convert_numpy_types(total_count), # Convert total_count
            "returned_patients": len(converted_patients),
            "limit": limit,
            "offset": offset,
            "patients": converted_patients
        }

        return create_response("success", f"Retrieved {len(converted_patients)} patients", patients_data)

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# ============================================================================
# API SERVER STARTUP
# ============================================================================

def start_api_server():
    """Start the FastAPI server with ngrok tunnel"""
    print("🚀 Starting Medical Records API Server...")

    # Kill any existing ngrok processes
    ngrok.kill()

    # Get ngrok authtoken from Colab secrets
    from google.colab import userdata
    NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
    if not NGROK_AUTH_TOKEN:
        print("❌ NGROK_AUTH_TOKEN not found in Colab secrets.")
        print("Please add your ngrok authtoken to Colab secrets.")
        print("Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken")
        return

    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

    # Start ngrok tunnel
    public_url = ngrok.connect(8000).public_url
    print(f"🌐 Public URL: {public_url}")
    print(f"📚 API Documentation: {public_url}/docs")
    print(f"🏠 Home Page: {public_url}/")


    # Start server in a new thread
    # This allows the Colab notebook to continue running while the server is active
    def run_server():
        uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

    thread = threading.Thread(target=run_server)
    thread.daemon = True # Allow the main thread to exit even if this thread is running
    thread.start()

    print("🏥 Medical Records API is now running!")
    print("Press Ctrl+C in the notebook to stop the server (may require interrupting the kernel)")


if __name__ == "__main__":
    start_api_server()

# For Colab: uncomment the next line to start the server
# start_api_server()

🚀 Starting Medical Records API Server...
🌐 Public URL: https://26221b41e215.ngrok-free.app
📚 API Documentation: https://26221b41e215.ngrok-free.app/docs
🏠 Home Page: https://26221b41e215.ngrok-free.app/
🏥 Medical Records API is now running!
Press Ctrl+C in the notebook to stop the server (may require interrupting the kernel)


In [13]:
import requests
import json

# Replace with your ngrok public URL displayed above
# Example: public_url = "https://your-random-subdomain.ngrok-free.app"
# Make sure to update this with the current public URL every time you restart the API server
public_url = "https://4a247adf0f65.ngrok-free.app" # Replace with your current ngrok URL

print(f"🚀 Testing API Endpoints at: {public_url}")
print("="*40)

# --- Test Endpoint: Get Patient Profile (e.g., P001) ---
patient_id_to_test = "P001"
get_profile_url = f"{public_url}/patients/{patient_id_to_test}"
print(f"\n➡️ GET {get_profile_url}")
try:
    response = requests.get(get_profile_url)
    response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
    profile_data = response.json()
    print("✅ Success!")
    # print(json.dumps(profile_data, indent=2)) # Uncomment to see the full response
    if profile_data and profile_data.get('data'):
        print(f"   Patient Name: {profile_data['data']['patient']['first_name']} {profile_data['data']['patient']['last_name']}")
        print(f"   Total Medications: {profile_data['data']['summary']['total_medications']}")
        print(f"   Total Visits: {profile_data['data']['summary']['total_visits']}")

except requests.exceptions.RequestException as e:
    print(f"❌ Error fetching patient profile: {e}")

# --- Test Endpoint: Smart Patient Search (e.g., "diabetes medication") ---
search_query = "diabetes medication"
search_url = f"{public_url}/patients/search/{search_query}"
print(f"\n➡️ GET {search_url}")
try:
    response = requests.get(search_url)
    response.raise_for_status()
    search_results = response.json()
    print("✅ Success!")
    if search_results and search_results.get('data'):
        print(f"   Query: '{search_results['data']['query']}'")
        print(f"   Total Results: {search_results['data']['total_results']}")
        if search_results['data']['patients']:
             print(f"   Sample Match: {search_results['data']['patients'][0]['first_name']} {search_results['data']['patients'][0]['last_name']}")

except requests.exceptions.RequestException as e:
    print(f"❌ Error searching patients: {e}")


# --- Test Endpoint: Risk Assessment (e.g., P001) ---
risk_url = f"{public_url}/patients/{patient_id_to_test}/risk"
print(f"\n➡️ GET {risk_url}")
try:
    response = requests.get(risk_url)
    response.raise_for_status()
    risk_data = response.json()
    print("✅ Success!")
    if risk_data and risk_data.get('data'):
        print(f"   Risk Level for {patient_id_to_test}: {risk_data['data']['risk_level']} (Score: {risk_data['data']['risk_score']})")

except requests.exceptions.RequestException as e:
    print(f"❌ Error fetching risk assessment: {e}")


# --- Test Endpoint: Drug Interaction Check (e.g., P001) ---
interactions_url = f"{public_url}/patients/{patient_id_to_test}/interactions"
print(f"\n➡️ GET {interactions_url}")
try:
    response = requests.get(interactions_url)
    response.raise_for_status()
    interactions_data = response.json()
    print("✅ Success!")
    if interactions_data and interactions_data.get('data'):
        print(f"   Interactions for {patient_id_to_test}: {interactions_data['data']['total_interactions']} found")
        if interactions_data['data']['interactions']:
             print(f"   Sample Interaction: {interactions_data['data']['interactions'][0]['warning']}")

except requests.exceptions.RequestException as e:
    print(f"❌ Error checking drug interactions: {e}")


# --- Test Endpoint: Database Stats ---
stats_url = f"{public_url}/stats/"
print(f"\n➡️ GET {stats_url}")
try:
    response = requests.get(stats_url)
    response.raise_for_status()
    stats_data = response.json()
    print("✅ Success!")
    if stats_data and stats_data.get('data'):
        print(f"   Total Patients: {stats_data['data']['database_overview']['total_patients']}")
        print(f"   Total Medications: {stats_data['data']['database_overview']['total_medications']}")

except requests.exceptions.RequestException as e:
    print(f"❌ Error fetching stats: {e}")


# --- Test Endpoint: Add New Patient (Example) ---
# Note: This will add a new patient every time you run it.
# You might want to comment this out after the first successful run.
# new_patient_url = f"{public_url}/patients/"
# new_patient_payload = {
#     "first_name": "Test",
#     "last_name": "Patient",
#     "date_of_birth": "1990-01-01",
#     "gender": "Female",
#     "phone": "123-456-7890",
#     "email": "test.patient@example.com",
#     "address": "123 Test St",
#     "insurance_id": "INS999999",
#     "emergency_contact": "Test Contact"
# }
# print(f"\n➡️ POST {new_patient_url}")
# try:
#     response = requests.post(new_patient_url, json=new_patient_payload)
#     response.raise_for_status()
#     create_patient_data = response.json()
#     print("✅ Success!")
#     if create_patient_data and create_patient_data.get('data'):
#          print(f"   New Patient ID: {create_patient_data['data']['patient_id']}")

# except requests.exceptions.RequestException as e:
#     print(f"❌ Error creating patient: {e}")


print("\n="*40)
print("🎉 API Testing Complete!")

🚀 Testing API Endpoints at: https://4a247adf0f65.ngrok-free.app

➡️ GET https://4a247adf0f65.ngrok-free.app/patients/P001
❌ Error fetching patient profile: 404 Client Error: Not Found for url: https://4a247adf0f65.ngrok-free.app/patients/P001

➡️ GET https://4a247adf0f65.ngrok-free.app/patients/search/diabetes medication
❌ Error searching patients: 404 Client Error: Not Found for url: https://4a247adf0f65.ngrok-free.app/patients/search/diabetes%20medication

➡️ GET https://4a247adf0f65.ngrok-free.app/patients/P001/risk
❌ Error fetching risk assessment: 404 Client Error: Not Found for url: https://4a247adf0f65.ngrok-free.app/patients/P001/risk

➡️ GET https://4a247adf0f65.ngrok-free.app/patients/P001/interactions
❌ Error checking drug interactions: 404 Client Error: Not Found for url: https://4a247adf0f65.ngrok-free.app/patients/P001/interactions

➡️ GET https://4a247adf0f65.ngrok-free.app/stats/
❌ Error fetching stats: 404 Client Error: Not Found for url: https://4a247adf0f65.ngrok-fre